In [1]:
using QuantumCumulants
using Symbolics
using SymbolicUtils
using ModelingToolkit, OrdinaryDiffEq
using MAT

In [2]:
file = matopen("/home/andreas/julia.mat")
gs = read(file, "coupl")
spins = read(file, "spins")
kappa = read(file, "kappa")
deltac = read(file, "deltac")
gperp = read(file, "gperp")
gpar = read(file, "gpar")

0.0006283185307179586

In [3]:
# Parameters
#IJulia.qtconsole()
N = length(gs) #number of atoms
M = 1
kappa_sym, n, Δ, i, q, gamma_par, etap, gamma_perp =
    cnumbers("κ ñ Δ_c I(t) Q(t) γ_{∥} η_p γ_{⊥}")

wf, G, kappa_f = cnumbers("Δ_f G κ_f")


g(j) = cnumbers(Symbol("g_{$j}"))[1]
Δs(j) = cnumbers(Symbol("Δ_{$j}"))[1]

# Hilbertspace
hf = FockSpace(:cavity)
hfilter = FockSpace(:cavity)
ha = ⊗([NLevelSpace(Symbol(:atom, i), 2) for i = 1:N]...)
h = hf ⊗ ha;

In [4]:
p = (
    kappa_sym => kappa,
    Δ => deltac,
    gamma_par => gpar,
    gamma_perp => gperp,
    i => 1,
    q => 0,
    n => 0,
    etap => 0,
    [g(i) => gs[i] for i = 1:N]...,
    [Δs(i) => 2 * pi * spins[i] for i = 1:N]...,
);

In [5]:
# Operators
a = Destroy(h, :a, 1)
sigma(i, j, k) = Transition(h, Symbol("σ_{$k}"), i, j, k + 1)
sigmaz(k) = sigma(2, 2, k) - sigma(1, 1, k)
sigmap(k) = sigma(2, 1, k);
sigmam(k) = sigma(1, 2, k);

In [6]:
H = Δ * a'a + 1im * i * (a' - a) + q * (a' + a)
for j = 1:N
    H += Δs(i) / 2 * sigmaz(j) + 1im * g(i) * (a' * sigmam(j) - a * sigmap(j))
end
#filter resonator
#H += wf*b'b+G*(a'b+b'a)

In [7]:
# Collapse operators

rates = Any[2*(n+1)*kappa, 2*n*kappa]
rates = append!(rates, [gamma_par * (n + 1) for i = 1:N])
rates = append!(rates, [n * gamma_par + etap for i = 1:N])
rates = append!(rates, [gamma_perp / 2 for i = 1:N])
#filter resonator
#rates = append!(rates, [2*kappa_f])



J = Any[a, a']
J = append!(J, [sigmam(i) for i = 1:N])
J = append!(J, [sigmap(i) for i = 1:N])
J = append!(J, [sigmaz(i) for i = 1:N])
#filter resonator
#J = append!(J, [b]);

In [ ]:
eqns = meanfield(a, H, J; rates = rates, order = M);

In [ ]:
@named sys = ODESystem(eqns)

u0 = zeros(ComplexF64, length(eqns))
prob = ODEProblem(sys, u0, (0.0, 15.0), p)
sol = solve(prob, RK4())